In [1]:
import numpy as np
import matplotlib.pyplot as plt
from numpy import math
import sympy as sym

$\textbf{1. Teoríco}$

dada la relación recursiva:  
$$
x_{n+1}=4x_{n}-x_{n}^{2}
$$
y la base $n=0$:
$$
x_{0}=4\sin^{2}(\theta)
$$
Se puede asuminar que la base esta dada por: 
$$
x_{0}=4\sin^{2}(2^{n}\theta)
$$
puesto que al ser $n=0$ el factor $2^{n}$ no afecta la relación inicial. Al remplazar en la relación recursiva se obtiene que:  
$$
x_{n+1}=4(4\sin^{2}(2^{n}\theta))-(4\sin^{2}(2^{n}\theta))^{2}
$$
por ende,
$$
x_{n+1}=16\sin^{2}(2^{n}\theta)-16\sin^{4}(2^{n}\theta)
$$
Al factorizar:
$$
x_{n+1}=16\sin^{2}(2^{n}\theta)(1-\sin^{2}(2^{n}\theta))
$$
$1-\sin^{2}(2^{n}\theta)$ puede ser remplazado por medio de identidades trigonométricas. Dando como resultado:

$$
x_{n+1}=16\sin^{2}(2^{n}\theta)(\cos^{2}(2^{n}\theta))
$$
Nuevamente aplicando identidades trigonometricas, se tiene que $2\sin(2^{n}\theta)\cos(2^{n}\theta)=\sin(2^{n+1}\theta)$ por ende al estar los valores al cuadrado se obtiene que el n-ésimo termino es: 

$$
x_{n+1}=4\sin^{2}(2^{n+1}\theta)
$$

Siendo así el término general queda: 

$$
x_{n}=4\sin^{2}(2^{n}\theta)
$$

$\textbf{2.  Gaus-Seidel}$

In [2]:
G1 = (
    lambda x, y, z: (1 + y + z) / 3,  
    lambda x, y, z: (3 + x - z) / 3, 
    lambda x, y, z: (7 - 2 * x - y) / 4 
)

In [3]:
def Gauss_seidel(f, r, h=1e-8, iter=1000):
    n = r.shape[0]  
    x = r.copy()  

    for k in range(iter):
        x_old = x.copy() 

        for i in range(n):
            x[i] = f[i](*x)  

        if np.linalg.norm(x - x_old, np.inf) < h:
            return x, k+1  
        
    return x, iter
    

r = np.array([1.,2.,0])

solucion, iteraciones=Gauss_seidel(G1,r)
solucion


array([1., 1., 1.])

$\textbf{3.  Clase para solucionar Gaus-Seidel y Jacobiano}$

In [4]:
def jacobi(G, r, h=1e-9, iter=1000):
    x, y, z = r
    x_new, y_new, z_new = x, y, z
    k = 0

    for i in range(iter):
        k += 1

        x_new = G[0](x, y, z)
        y_new = G[1](x, y, z)
        z_new = G[2](x, y, z)

        # Verifica la convergencia
        if abs(x_new - x) < h and abs(y_new - y) < h and abs(z_new - z) < h:
            break

        x, y, z = x_new, y_new, z_new

    return x_new, y_new, z_new, k

jacobi(G1,r)

(1.0000000000254434, 1.0000000002613536, 0.9999999997626395, 20)

In [5]:
class SistemaLineal:
    def __init__(self, funciones):
        self.funciones = funciones

    def jacobi(self):
        n = np.zeros(len(self.funciones))  
        x_new, y_new, z_new, iteraciones = jacobi(self.funciones, n)  
        solucion = (x_new, y_new, z_new)  
        print(f"Jacobi necesitó {iteraciones} iteraciones, \ny la solución es: ")
        return solucion

    def gauss_seidel(self):
        n = np.zeros(len(self.funciones)) 
        solucion, iteraciones = Gauss_seidel(self.funciones, n)
        print(f"Gauss-Seidel necesitó {iteraciones} iteraciones, \ny la solución es:")
        return solucion


sistema = SistemaLineal(G1)

solucion_jacobi = sistema.jacobi()
print(solucion_jacobi)

solucion_gs = sistema.gauss_seidel()
print(solucion_gs)


Jacobi necesitó 21 iteraciones, 
y la solución es: 
(0.9999999999702068, 0.999999999838595, 1.0000000001564002)
Gauss-Seidel necesitó 15 iteraciones, 
y la solución es:
[1. 1. 1.]


$\textbf{4.  Algoritmo que realice la multiplicación de dos matrices}$

In [15]:
A=np.array([[1,0,0],[5,1,0],[-2,3,1]])
B=np.array([[4,-2,1],[0,3,7],[0,0,2]])

array([[0., 0.],
       [0., 0.]])

In [20]:
def multiplicar_matriz(A,B):
    n=len(A)
    m=len(B[0])
    c=np.zeros((n,m))

    for i in range(len(A)):
        for j in range(len(B)):
            for k in range(len(A[0])):
                c[i][j]+=A[i][k]*B[k][j]
    return c

multiplicar_matriz(A,B)

array([[ 4., -2.,  1.],
       [20., -7., 12.],
       [-8., 13., 21.]])

$\textbf{5. sustitución hacia adelante}$

Dado el sistema lineal:
$$\begin{pmatrix}
a_{11} & a_{12} & a_{13} & .. & a_{1n} \\
a_{21} & a_{22} & a_{23} & .. & a_{2n} \\
.. & .. & .. & .. & .. \\
.. & .. & .. & .. & .. \\
a_{n1} & a_{n2} & a_{n3} & .. & a_{nn} 
\end{pmatrix} 
 \begin{pmatrix}
x_1 \\
x_2 \\
x_3 \\
.. \\
x_n 
\end{pmatrix}  
    =
\begin{pmatrix}
b_1 \\
b_2 \\
b_3 \\
.. \\
b_n 
\end{pmatrix}
 $$

esta puede ser aumentada de la forma aumentada  \( A | b \) donde \( A \) es la matriz de coeficientes y \( b \) es el vector de resultados. Se presenta de la siguiente manera:

$$
\begin{pmatrix}
a_{11} & a_{12} & a_{13} & \cdots & a_{1n} & | & b_1 \\
a_{21} & a_{22} & a_{23} & \cdots & a_{2n} & | & b_2 \\
\vdots & \vdots & \vdots & \vdots & \vdots & | & \vdots \\
a_{n1} & a_{n2} & a_{n3} & \cdots & a_{nn} & | & b_n 
\end{pmatrix}
$$


Para realizar la sustitución hacia adelante, comenzamos despejando la primera variable $x_1$  de la primera ecuación:

$$
x_1 = \frac{b_1 - (a_{12} x_2 + a_{13} x_3 + \cdots + a_{1n} x_n)}{a_{11}}
$$


Posteriormente se despeja $x_2$ en la segunda ecuación:

$$
a_{21} x_1 + a_{22} x_2 + a_{23} x_3 + \cdots + a_{2n} x_n = b_2
$$

Al hacer la sustitución en $x_1$:

$$
a_{21} \left( \frac{b_1 - (a_{12} x_2 + a_{13} x_3 + \cdots + a_{1n} x_n)}{a_{11}} \right) + a_{22} x_2 + a_{23} x_3 + \cdots + a_{2n} x_n = b_2
$$

Reorganizando, obtenemos la expresión para $x_2$:

$$
x_2 = \frac{b_2 - a_{21} \left( \frac{b_1 - (a_{12} x_2 + a_{13} x_3 + \cdots + a_{1n} x_n)}{a_{11}} \right) - (a_{23} x_3 + \cdots + a_{2n} x_n)}{a_{22}}
$$

Este proceso continúa para las demás variables hasta llegar a $x_n$.

Podemos generalizar este procedimiento para cualquier $i$:

$$
x_i = \frac{b_i - \sum_{j=0}^{i-1} a_{ij} x_j}{a_{ii}}
$$

La sustitución hacia adelante permite encontrar las soluciones de un sistema de ecuaciones lineales de manera sistemática. Al comenzar desde la primera incógnita y trabajar hacia la última, se asegura que cada variable se resuelva en términos de las anteriores, facilitando así la solución del sistema.


$\textbf{6. sustitución hacia atrás}$


La sustitución hacia atrás se utiliza en el método de eliminación de Gauss, para iniciar supongamos que se a llevado a una expresión triangular superior

$$
\begin{pmatrix}
1 & a_{12} & a_{13} & \cdots & a_{1n} & | & b_1 \\
0 & 1 & a_{23} & \cdots & a_{2n} & | & b_2' \\
0 & 0 & 1 & \cdots & a_{3n} & | & b_3'' \\
\vdots & \vdots & \vdots & \vdots & \vdots & | & \vdots \\
0 & 0 & 0 & \cdots & 1 & | & b_n^{(n)}
\end{pmatrix}
$$

Aquí, la última fila representa la ecuación $ 1 \cdot x_n = b_n^{(n)} $, donde $b_n^{(n)} $ es un valor calculado a partir de $ b $.

La solución para la última variable es: 

$$
x_n = b_n^{(n)}
$$

Al desarrolar la sustitución hacia atrás a partir de lo anterior, primero se debe encontrar $ x_{n-1} $, para ello se utiliza la penúltima ecuación:

$$
0 \cdot x_n + 1 \cdot x_{n-1} + a_{(n-1)n} \cdot x_n = b_{n-1}^{(n-1)}
$$

Esto se puede reescribir como:

$$
x_{n-1} = b_{n-1}^{(n-1)} - a_{(n-1)n} \cdot x_n
$$

Al continuar con las sustitución hacia atrás utilizando cada ecuación para despejar la siguiente variable, como por ejemplo:
$$
x_{n-2} = b_{n-2}^{(n-2)} - a_{(n-2)(n-1)} \cdot x_{n-1} - a_{(n-2)n} \cdot x_n
$$

De este modo se lleva a que para $x_1$ equivale a: 

$$
x_1 = b_1^{(1)} - a_{12} \cdot x_2 - a_{13} \cdot x_3 - \ldots - a_{1n} \cdot x_n
$$

Este proceso de sustitución hacia atrás puede generalizarse para cualquier $ i $ de la siguiente manera:

$$
x_i = b_i^{(i)} - \sum_{j=i+1}^{n} a_{ij} \cdot x_j
$$

Donde $ a_{ij} $ son los coeficientes que multiplican las incógnitas en la $i $-ésima ecuación.


El método de sustitución hacia atrás permite encontrar las soluciones de un sistema de ecuaciones lineales de manera sistemática. Al comenzar desde la última incógnita y trabajar hacia la primera, de esto modo se comprueba que cada variable es resuelta en términos de las que ya han sido calculadas, lo que facilita el proceso de resolución y asegura la consistencia del sistema.


$\textbf{8. Descomposición LU}$

In [37]:
n=3
A=np.array([[4,-2,1],[20,-7,12],[-8,13,17]])

def descomposición_LU(A):
    n=len(A)
    U=np.zeros([n,n])
    L=np.zeros([n,n])
    for i in range(0,n):
        for j in range(0,n):
            U[i,j]=A[i,j]

    #Hacer tringular superior
    for k in range(0,n):
        for r in range(0,n):
            if (k==r):
                L[k,r]=1
            if (k<r):
                f=(A[r,k]/A[k,k])
                L[r,k]=f
                for c in range(0,n):
                    A[r,c]=A[r,c]-f*A[k,c]
                    U[r,c]=A[r,c]
    return L, U

L,U=descomposición_LU(A)
print(L)
print(U)

[[ 1.  0.  0.]
 [ 5.  1.  0.]
 [-2.  3.  1.]]
[[ 4. -2.  1.]
 [ 0.  3.  7.]
 [ 0.  0. -2.]]


In [40]:
#Un método de comprobación
print(multiplicar_matriz(L,U))

a=np.dot(L,U)
print(a)

[[ 4. -2.  1.]
 [20. -7. 12.]
 [-8. 13. 17.]]
[[ 4. -2.  1.]
 [20. -7. 12.]
 [-8. 13. 17.]]
